# CS336 Assignment 2 - 易错点笔记

## 1. pandas 自动生成表格

作业中会有大量实验数据需要整理成表格，手动写 LaTeX/Markdown 很繁琐。

- `df.to_markdown()` → 用于 Jupyter Notebook、GitHub README
- `df.to_latex()` → 用于 LaTeX 论文、PDF 报告

In [ ]:
import pandas as pd

data = {
    "Size": ["small", "medium", "large", "xl", "2.7B"],
    "d_model": [768, 1024, 1280, 1600, 2560],
    "d_ff": [3072, 4096, 5120, 6400, 10240],  # d_ff = 4 * d_model
    "num_layers": [12, 24, 36, 48, 32],
    "num_heads": [12, 16, 20, 25, 32]
}

df = pd.DataFrame(data)
print(df.to_markdown(index=False))

---

## 2. CUDA Benchmarking 易错点

### 易错点 1: CUDA 异步执行

**错误理解**: `end - start` 测量的是 GPU 实际计算时间  
**正确理解**: CUDA 是异步的，CPU 发出指令后立刻返回，不等 GPU 完成

```python
# ❌ 错误：测量的是 CPU 发出指令的时间，不是 GPU 计算时间
start = time.time()
output = model(input)
end = time.time()
```

### 易错点 2: torch.cuda.synchronize() 的作用

**错误理解**: 让 CPU 在等待 GPU 时可以做其他事  
**正确理解**: 强制 CPU 停下来等待，直到 GPU 完成所有任务

```python
# ✅ 正确：synchronize 后 end-start 才是真正的 GPU 计算时间
start = time.time()
output = model(input)
torch.cuda.synchronize()  # CPU 在这里等待 GPU 完成
end = time.time()
```

### 易错点 3: Warm-up 预热的含义

**错误理解**: 预热是学习率预热，防止学习率过大  
**正确理解**: 基准测试预热 ≠ 学习率预热！

**基准测试预热的原因**:
1. **JIT 编译**: 第一次运行时需要编译 CUDA kernel
2. **内存分配**: 第一次运行时需要分配 GPU 内存
3. **缓存冷启动**: 第一次运行时缓存是空的

所以第一次运行通常比后续运行**慢很多**，需要排除在计时之外。

In [ ]:
# Benchmark 正确模板
import torch
import timeit

def benchmark_model(model, input_data, warmup_steps=5, measure_steps=10):
    """正确的 GPU benchmarking 模板"""
    
    # 1. Warm-up: 不计时
    for _ in range(warmup_steps):
        output = model(input_data)
        torch.cuda.synchronize()
    
    # 2. 测量: 每次都要 synchronize
    times = []
    for _ in range(measure_steps):
        start = timeit.default_timer()
        output = model(input_data)
        torch.cuda.synchronize()  # 关键！
        end = timeit.default_timer()
        times.append(end - start)
    
    return times